In [ ]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
#spark_version = 'spark-3.4.0'
spark_version='spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

#Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/SPARK_VERSION/SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark
!pip install --upgrade pip

Hit:1 https://repos.azul.com/zulu/deb stable InRelease
Hit:2 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease

W: https://repos.azul.com/zulu/deb/dists/stable/InRelease: Key is stored in legacy trusted.gpg keyring (/etc/apt/trusted.gpg), see the DEPRECATION section in apt-key(8) for details.
tar: spark-3.4.0-bin-hadoop3.tgz: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now
DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: distro-info 1.1build1 has a non-stan

In [ ]:
#import findspark and initialise
import findspark
findspark.init()

In [ ]:
#import packages
from pyspark.sql import SparkSession
import time

#Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [ ]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales.csv"

In [ ]:
# File location and type
file_location = "/FileStore/tables/home_sales.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

In [ ]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView("home_sales")
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [ ]:

#Query that shows the top 10 rows of the temporary view.

results = spark.sql("""SELECT * FROM home_sales""")

results.show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [ ]:
# 3. what is the average price for a four bedroom house sold in each year rounded to two decimal places?

results = spark.sql("""
    SELECT 
        date_built,
        ROUND(AVG(price), 2) as average_price
    FROM 
        home_sales
    WHERE
        bedrooms = 4
    GROUP BY date_built
    ORDER BY date_built DESC;
""")

results.show()

+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2017|    296576.69|
|      2016|    296050.24|
|      2015|    307908.86|
|      2014|    299073.89|
|      2013|    299999.39|
|      2012|    298233.42|
|      2011|     302141.9|
|      2010|    296800.75|
+----------+-------------+



In [ ]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
results = spark.sql("""
    SELECT 
    YEAR(date_built) as year_sold,
    ROUND(AVG(price),2) as average_price
    FROM 
        home_sales
    WHERE
        bedrooms = 3 AND bathrooms = 3
    GROUP BY date_built
    ORDER BY date_built DESC;""")
results.show()

+---------+-------------+
|year_sold|average_price|
+---------+-------------+
|     2017|    292676.79|
|     2016|    290555.07|
|     2015|     288770.3|
|     2014|    290852.27|
|     2013|    295962.27|
|     2012|    293683.19|
|     2011|    291117.47|
|     2010|    292859.62|
+---------+-------------+



In [ ]:
#5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
#and are greater than or equal to 2,000 square feet rounded to two decimal places?
results = spark.sql("""
SELECT 
    YEAR(date_built) as year_sold,
    ROUND(AVG(price),2) as average_price
FROM 
    home_sales
WHERE
    bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
GROUP BY
    date_built
ORDER BY date_built DESC;""")
results.show()

+---------+-------------+
|year_sold|average_price|
+---------+-------------+
|     2017|    280317.58|
|     2016|     293965.1|
|     2015|    297609.97|
|     2014|    298264.72|
|     2013|    303676.79|
|     2012|    307539.97|
|     2011|    276553.81|
|     2010|    285010.22|
+---------+-------------+



In [ ]:
#6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
#or equal to $350,000? 
start_time = time.time()

results = spark.sql("""
SELECT 
    view,
    ROUND(avg(price),2) as average_price
FROM 
    home_sales
GROUP BY view
HAVING
    average_price >= 35000
ORDER BY view DESC;
""")
print("--- %s seconds ---" % (time.time() - start_time))

results.show()

--- 0.01686692237854004 seconds ---
+----+-------------+
|view|average_price|
+----+-------------+
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|   9|    282001.65|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
+----+-------------+
only showing top 20 rows



In [ ]:
#7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [ ]:
#8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [ ]:
#9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.
start_time = time.time()

results = spark.sql("""
SELECT 
    view,
    ROUND(avg(price),2) as average_price
FROM 
    home_sales
GROUP BY view
HAVING
    average_price >= 35000
ORDER BY average_price DESC;
""")
print("--- %s seconds ---" % (time.time() - start_time))

results.show()

--- 0.01987934112548828 seconds ---
+----+-------------+
|view|average_price|
+----+-------------+
|  91|   1137372.73|
|  97|   1129040.15|
|  84|   1117233.13|
|  75|   1114042.94|
|  89|   1107839.15|
|  78|   1080649.37|
|  77|   1076205.56|
|  87|    1072285.2|
|  86|   1070444.25|
|  82|    1063498.0|
|  90|   1062654.16|
|  99|   1061201.42|
|  76|   1058802.78|
|  85|   1056336.74|
|  95|    1054325.6|
|  98|   1053739.33|
|  81|   1053472.79|
|  83|   1033965.93|
|  94|    1033536.2|
|  88|   1031719.35|
+----+-------------+
only showing top 20 rows



In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy("date_built").parquet("parquet_home_sales", mode="overwrite")

In [ ]:
# 11. Read the parquet formatted data.
spark.read.parquet('dbfs:/parquet_home_sales') 

DataFrame[id: string, date: string, price: string, bedrooms: string, bathrooms: string, sqft_living: string, sqft_lot: string, floors: string, waterfront: string, view: string, date_built: int]

In [ ]:
# 12. Create a temporary table for the parquet data.
#p_df.createOrReplaceTempView(df)

p_df.createOrReplaceTempView('parquet_home_sales')

In [ ]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version. 

start_time = time.time()

results = spark.sql("""
SELECT
    ROUND(avg(price),2) as average_price
FROM 
    home_sales
GROUP BY view
HAVING
    average_price >= 35000
ORDER BY average_price DESC;
""")
print("--- %s seconds ---" % (time.time() - start_time))

results.show()

--- 0.014125823974609375 seconds ---
+-------------+
|average_price|
+-------------+
|   1137372.73|
|   1129040.15|
|   1117233.13|
|   1114042.94|
|   1107839.15|
|   1080649.37|
|   1076205.56|
|    1072285.2|
|   1070444.25|
|    1063498.0|
|   1062654.16|
|   1061201.42|
|   1058802.78|
|   1056336.74|
|    1054325.6|
|   1053739.33|
|   1053472.79|
|   1033965.93|
|    1033536.2|
|   1031719.35|
+-------------+
only showing top 20 rows



In [ ]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [ ]:
# 15. Check if the home_sales is no longer cached
if spark.catalog.isCached("home_sales"):
  print("a table is till cached")
else:
  print("all clear") 

all clear
